In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!pip install thop

In [0]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.backends.cudnn as cudnn
import math
import time
import torch.nn.init as init
import csv
import shutil
import pathlib
from os import remove
from os.path import isfile
from collections import OrderedDict

from google.colab import files


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#config
dataset = 'cifar100'
workers = 8
batch_size = 128
epochs = 200
learning_rate = 0.1
momentum = 0.9
weight_decay = 5e-4
print_freq = 50
ckpt = '/content/drive/My Drive/Coding/nowdoing2/SQUEEZENET_result/checkpoint/'


def load_model(model, ckpt_file):
    checkpoint = torch.load(ckpt_file, map_location=lambda storage, loc: storage)
    try:
        model.load_state_dict(checkpoint['model'])
    except:
        # create new OrderedDict that does not contain `module.`
        new_state_dict = OrderedDict()
        for k, v in checkpoint['model'].items():
            if k[:7] == 'module.':
                name = k[7:] # remove `module.`
            else:
                name = k[:]
            new_state_dict[name] = v

        model.load_state_dict(new_state_dict)

    return checkpoint

def save_model(state, epoch, is_best):
    dir_ckpt = pathlib.Path('/content/drive/My Drive/Coding/nowdoing2/SQUEEZENET_result/checkpoint/')
    dir_path = dir_ckpt / dataset
    dir_path.mkdir(parents=True, exist_ok=True)

    model_file = dir_path / 'ckpt_epoch_{}.pth'.format(epoch)
    torch.save(state, model_file)

    if is_best:
        shutil.copyfile(model_file, dir_path / 'ckpt_best.pth')


class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)


class ProgressMeter(object):
    def __init__(self, num_batches, *meters, prefix=""):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = prefix

    def print(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print('\t'.join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'


normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

transform = transforms.Compose([
    transforms.Pad(4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32),
    transforms.ToTensor(),
    #normalize,
    ])

# CIFAR-10 dataset
train_dataset = torchvision.datasets.CIFAR100(root='../../data/',
                                             train=True, 
                                             transform=transform,
                                             download=True)

test_dataset = torchvision.datasets.CIFAR100(root='../../data/',
                                            train=False,
                                            #transform=transforms.Compose([
                                            transform=transforms.ToTensor(),
                                            #normalize,                                                                
                                            #])
)

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

val_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)
# 3x3 convolution
def conv3x3(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3, 
                     stride=stride, padding=1, bias=False)




__all__ = ['SqueezeNet', 'squeezenet1_0', 'squeezenet1_1']

model_urls = {
    'squeezenet1_0': 'https://download.pytorch.org/models/squeezenet1_0-a815701f.pth',
    'squeezenet1_1': 'https://download.pytorch.org/models/squeezenet1_1-f364aa15.pth',
}


class Fire(nn.Module):

    def __init__(self, inplanes, squeeze_planes,
                 expand1x1_planes, expand3x3_planes):
        super(Fire, self).__init__()
        self.inplanes = inplanes
        self.squeeze = nn.Conv2d(inplanes, squeeze_planes, kernel_size=1)
        self.squeeze_activation = nn.ReLU(inplace=True)
        self.expand1x1 = nn.Conv2d(squeeze_planes, expand1x1_planes,
                                   kernel_size=1)
        self.expand1x1_activation = nn.ReLU(inplace=True)
        self.expand3x3 = nn.Conv2d(squeeze_planes, expand3x3_planes,
                                   kernel_size=3, padding=1)
        self.expand3x3_activation = nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.squeeze_activation(self.squeeze(x))
        out = torch.cat([
              self.expand1x1_activation(self.expand1x1(x)),
              self.expand3x3_activation(self.expand3x3(x))
          ], 1)
        #print("Output Size: {}".format(out))
        return out


class SqueezeNet(nn.Module):

    def __init__(self, version='1_0', num_classes=100):
        super(SqueezeNet, self).__init__()
        self.num_classes = num_classes
        if version == '1_0':
            self.features = nn.Sequential(
                nn.Conv2d(3, 96, kernel_size=3, stride=1),
                nn.ReLU(inplace=True),
                #nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True),
                Fire(96, 16, 64, 64),
                Fire(128, 16, 64, 64),
                Fire(128, 32, 128, 128),
                nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True),
                Fire(256, 32, 128, 128),
                Fire(256, 48, 192, 192),
                Fire(384, 48, 192, 192),
                Fire(384, 64, 256, 256),
                nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True),
                Fire(512, 64, 256, 256),
            )
        elif version == '1_1':
            self.features = nn.Sequential(
                nn.Conv2d(3, 64, kernel_size=3, stride=2),
                nn.ReLU(inplace=True),
                nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True),
                Fire(64, 16, 64, 64),
                Fire(128, 16, 64, 64),
                nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True),
                Fire(128, 32, 128, 128),
                Fire(256, 32, 128, 128),
                nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True),
                Fire(256, 48, 192, 192),
                Fire(384, 48, 192, 192),
                Fire(384, 64, 256, 256),
                Fire(512, 64, 256, 256),
            )
        else:
            # FIXME: Is this needed? SqueezeNet should only be called from the
            # FIXME: squeezenet1_x() functions
            # FIXME: This checking is not done for the other models
            raise ValueError("Unsupported SqueezeNet version {version}:"
                             "1_0 or 1_1 expected".format(version=version))

        # Final convolution is initialized differently from the rest
        final_conv = nn.Conv2d(512, self.num_classes, kernel_size=1)
        self.classifier = nn.Sequential(
            nn.Dropout(p=0.5),
            final_conv,
            nn.ReLU(inplace=True),
            #nn.AdaptiveAvgPool2d((1, 1))
        )

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                if m is final_conv:
                    init.normal_(m.weight, mean=0.0, std=0.01)
                else:
                    init.kaiming_uniform_(m.weight)
                if m.bias is not None:
                    init.constant_(m.bias, 0)

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return torch.flatten(x, 1)


def _squeezenet(version, pretrained, progress, **kwargs):
    model = SqueezeNet(version, **kwargs)
    if pretrained:
        arch = 'squeezenet' + version
        state_dict = load_state_dict_from_url(model_urls[arch],
                                              progress=progress)
        model.load_state_dict(state_dict)
    return model


def squeezenet1_0(pretrained=False, progress=True, **kwargs):
    r"""SqueezeNet model architecture from the `"SqueezeNet: AlexNet-level
    accuracy with 50x fewer parameters and <0.5MB model size"
    <https://arxiv.org/abs/1602.07360>`_ paper.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _squeezenet('1_0', pretrained, progress, **kwargs)


def squeezenet1_1(pretrained=False, progress=True, **kwargs):
    r"""SqueezeNet 1.1 model from the `official SqueezeNet repo
    <https://github.com/DeepScale/SqueezeNet/tree/master/SqueezeNet_v1.1>`_.
    SqueezeNet 1.1 has 2.4x less computation and slightly fewer parameters
    than SqueezeNet 1.0, without sacrificing accuracy.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _squeezenet('1_1', pretrained, progress, **kwargs)

def adjust_learning_rate(optimizer, epoch, lr):
    """Sets the learning rate, decayed rate of 0.1 every epoch"""
    #if epoch >= 150:
    #  lr = 0.01
    #if epoch >=80:
    #  lr = 0.001
    #if epoch >=120:
    #  lr = 0.0001

    for param_group in optimizer.param_groups:
        param_group['lr'] = lr


def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].view(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res



best_acc1 = 0


model = squeezenet1_0()
print(model)
from thop import profile
input = torch.randn(1, 3, 32, 32)
flops, params = profile(model, inputs=(input, ))
from thop import clever_format
macs, params = clever_format([flops, params], "%.3f")
print("MACS: {}".format(macs))
print("Params: {}".format(params))


print(model)

criterion = nn.CrossEntropyLoss()
#optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate,
                            weight_decay=weight_decay, momentum=0.4,
                            nesterov=True)

start_epoch = 0


model = model.to(device)
criterion = criterion.to(device)



ckpt_dir = pathlib.Path('checkpoint')
ckpt_file = ckpt_dir / dataset / ckpt

train_time = 0.0
validate_time = 0.0
lr = learning_rate
max_grad_norm = 2.0

def train(train_loader, **kwargs):
    epoch = kwargs.get('epoch')
    model = kwargs.get('model')
    criterion = kwargs.get('criterion')
    optimizer = kwargs.get('optimizer')

    batch_time = AverageMeter('Time', ':6.3f')
    data_time = AverageMeter('Data', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(len(train_loader), batch_time, data_time,
                             losses, top1, top5, prefix="Epoch: [{}]".format(epoch))

    # switch to train mode
    model.train()

    end = time.time()
    running_loss = 0.0
    for i, (input, target) in enumerate(train_loader):
        # measure data loading time
        data_time.update(time.time() - end)

        input = input.to(device)
        target = target.to(device)
        #if args.cuda:
        #    target = target.cuda(non_blocking=True)

        # compute output
        output = model(input)
        loss = criterion(output, target)

        # measure accuracy and record loss
        acc1, acc5 = accuracy(output, target, topk=(1, 5))
        losses.update(loss.item(), input.size(0))
        top1.update(acc1[0], input.size(0))
        top5.update(acc5[0], input.size(0))

        # compute gradient and do SGD step.
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)

        running_loss += loss.item()

        if i % print_freq == 0:
            progress.print(i)

        end = time.time()

    print('====> Acc@1 {top1.avg:.3f} Acc@5 {top5.avg:.3f}'
          .format(top1=top1, top5=top5))
    epoch_loss = running_loss / len(train_loader)
    print('====> Epoch loss {:.3f}'.format(epoch_loss))

    return top1.avg, top5.avg, epoch_loss


def validate(val_loader, model, criterion):
    batch_time = AverageMeter('Time', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(len(val_loader), batch_time, losses, top1, top5,
                             prefix='Test: ')

    # switch to evaluate mode
    model.eval()
    total_loss = 0.0

    with torch.no_grad():
        end = time.time()
        for i, (input, target) in enumerate(val_loader):
            #if args.cuda:
            #    target = target.cuda(non_blocking=True)
            input = input.to(device)
            target = target.to(device)

            # compute output
            output = model(input)
            loss = criterion(output, target)

            # measure accuracy and record loss
            acc1, acc5 = accuracy(output, target, topk=(1, 5))
            losses.update(loss.item(), input.size(0))
            top1.update(acc1[0], input.size(0))
            top5.update(acc5[0], input.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            total_loss += loss.item()

            if i % print_freq == 0:
                progress.print(i)

            end = time.time()

        print('====> Acc@1 {top1.avg:.3f} Acc@5 {top5.avg:.3f}'
              .format(top1=top1, top5=top5))
        total_loss = total_loss/ len(val_loader)

    return top1.avg, top5.avg, loss.item()

start_time = time.time()


result_epoch = []
result_lr = []
result_train_avgtime = []
result_train_avgloss = []
result_train_avgtop1acc = []
result_train_avgtop5acc = []
result_test_avgtime = []
result_test_avgtop1acc = []
result_test_avgtop5acc = []

"""
Results
Epoch
learning rate
Training average loss per epoch
Training average time per epoch
Training average top1 accuracy per epoch
Training average top5 accuracy per epoch
Training average top1 error per epoch
Training average top5 error per epoch
Validation average loss per epoch
Validation average time per epoch
Validation average top1 accuracy per epoch
Validation average top5 accuracy per epoch
Validation average top1 error per epoch
Validation average top5 error per epoch
Best test accuracy
"""

for epoch in range(start_epoch, epochs):
  adjust_learning_rate(optimizer, epoch, lr)

  print('\n==> Epoch: {}, lr = {}'.format(
        epoch, optimizer.param_groups[0]["lr"]))
  result_epoch.append(epoch)
  result_lr.append(optimizer.param_groups[0]["lr"])

  # train for one epoch
  print('===> [ Training ]')
  start_time = time.time()
  acc1_train, acc5_train, avgloss_train = train(train_loader,
      epoch=epoch, model=model,
      criterion=criterion, optimizer=optimizer)
  result_train_avgloss.append(avgloss_train)
  result_train_avgtop1acc.append(round(acc1_train.item(),2))
  result_train_avgtop5acc.append(round(acc5_train.item(),2))
  elapsed_time = time.time() - start_time
  train_time += elapsed_time
  print('====> {:.2f} seconds to train this epoch\n'.format(
      elapsed_time))
  result_train_avgtime.append(elapsed_time)

  # evaluate on validation set
  print('===> [ Validation ]')
  start_time = time.time()
  acc1_valid, acc5_valid, avgloss_valid = validate(val_loader, model, criterion)
  result_test_avgtop1acc.append(acc1_valid.item())
  result_test_avgtop5acc.append(acc5_valid.item())
  elapsed_time = time.time() - start_time
  validate_time += elapsed_time
  print('====> {:.2f} seconds to validate this epoch\n'.format(
      elapsed_time))
  result_test_avgtime.append(elapsed_time)

  import pandas as pd
  """
  print(result_epoch)
  print(result_lr)
  print(result_train_avgloss)
  print(result_train_avgtop1acc)
  print(result_train_avgtop5acc)
  print(result_test_avgtop1acc)
  print(result_test_avgtop5acc)
  """
  df = pd.DataFrame({
      'Epoch': result_epoch,
      'Learning rate': result_lr,
      'Training avg loss': result_train_avgloss,
      'Training avg top1 acc': result_train_avgtop1acc,
      'Training avg top5 acc': result_train_avgtop5acc,
      'Test avg top1 acc': result_test_avgtop1acc,
      'Test avg top5 acc': result_test_avgtop5acc,
  })
  #df.to_csv('resnet_result.csv')

  # remember best Acc@1 and save checkpoint
  is_best = acc1_valid > best_acc1
  best_acc1 = max(acc1_valid, best_acc1)
  state = {'epoch': epoch + 1,
            'model': model.state_dict(),
            'optimizer': optimizer.state_dict()}
  if (epoch + 1) % 20 == 0:
    save_model(state, epoch, is_best)
    df.to_csv('/content/drive/My Drive/Coding/nowdoing2/SQUEEZENET_result/squeeze_result.csv')

avg_train_time = train_time / (epochs-start_epoch)
avg_valid_time = validate_time / (epochs-start_epoch)
total_train_time = train_time + validate_time
print('====> average training time per epoch: {:,}m {:.2f}s'.format(
    int(avg_train_time//60), avg_train_time%60))
print('====> average validation time per epoch: {:,}m {:.2f}s'.format(
    int(avg_valid_time//60), avg_valid_time%60))
print('====> training time: {}h {}m {:.2f}s'.format(
    int(train_time//3600), int((train_time%3600)//60), train_time%60))
print('====> validation time: {}h {}m {:.2f}s'.format(
    int(validate_time//3600), int((validate_time%3600)//60), validate_time%60))
print('====> total training time: {}h {}m {:.2f}s'.format(
    int(total_train_time//3600), int((total_train_time%3600)//60), total_train_time%60))

elapsed_time = time.time() - start_time
print('====> total time: {}h {}m {:.2f}s'.format(
      int(elapsed_time//3600), int((elapsed_time%3600)//60), elapsed_time%60))


# Test the model
model.eval()
result_test_acc = 0.0
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in val_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    result_test_acc = 100 * correct / total
    print('Accuracy of the model on the test images: {} %'.format(100 * correct / total))

acc = []
acc.append(result_test_acc)
df2 = pd.DataFrame({
      'Accuracy': acc,
})
df2.to_csv('resnet_accuracy.csv')
#files.download('resnet_result.csv')
#files.download('resnet_accuracy.csv')


Extracting ../../data/cifar-100-python.tar.gz to ../../data/
SqueezeNet(
  (features): Sequential(
    (0): Conv2d(3, 96, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU(inplace=True)
    (2): Fire(
      (squeeze): Conv2d(96, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (3): Fire(
      (squeeze): Conv2d(128, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (4): Fire(
      (squeeze): Conv2d(

KeyboardInterrupt: ignored